# Download Pretrained Model

In [0]:
!pip install contractions
!pip install PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# download pre-trained model
# you can replace the id with id of file you want to access
model_id = "1vXsf0DI8HKGuqYFIGIrvG-73KzZPgz5h"
downloaded = drive.CreateFile({'id':model_id})
downloaded.GetContentFile('wiki_Animal.model')

2020-04-23 13:42:47,596 : WARNING : file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFo

# Provide Callback

In [0]:
from time import time

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from gensim.models.callbacks import CallbackAny2Vec
class Callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 1

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 1:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

# Load Pretrained Model

In [0]:
from gensim.models import Word2Vec
model = Word2Vec.load("wiki_Animal.model")
print(model)

2020-04-23 14:10:42,791 : INFO : loading Word2Vec object from wiki_Animal.model
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-04-23 14:10:43,313 : INFO : loading wv recursively from wiki_Animal.model.wv.* with mmap=None
2020-04-23 14:10:43,314 : INFO : setting ignored attribute vectors_norm to None
2020-04-23 14:10:43,318 : INFO : loading vocabulary recursively from wiki_Animal.model.vocabulary.* with mmap=None
2020-04-23 14:10:43,322 : INFO : loading trainables recursively from wiki_Animal.model.trainables.* with mmap=None
2020-04-23 14:10:43,323 : INFO : setting ignored attribute cum_table to None
2020-04-23 14:10:43,324 : INFO : loaded wiki_Animal.model


Word2Vec(vocab=23190, size=300, alpha=0.005)


In [0]:
old_vocab = list(model.wv.vocab)

# Gather and Preprocess New Data

In [0]:
from elang.word2vec.builder import build_from_wikipedia
build_from_wikipedia(
    slug = "Cuteness", # scrape with query
    levels = 1, # levels of articles
    lang = "en", # english language
    model = False, # don't create model
    save = True) # save to txt file

Level 1 Queried so far: ['Cranium', 'Anteroposterior', 'Tardigrades', 'Birth_control_pill', 'London_University', 'Sweet_Lolita', 'Cuteness', 'Supraorbital_ridge', 'Human_development_(biology)', 'Socially_accepted', 'Heart_(symbol)', 'Domestication_of_dogs'] 

Article content successfully saved to wikipedia_branch_Cuteness_1.txt


In [0]:
wiki_file = open("/content/corpus/txt/wikipedia_branch_Cuteness_1.txt", "r")
wiki_text = wiki_file.readlines()
wiki_text[:5]

['  The skull is a bony structure that forms the head in vertebrates. It supports the structures of the face and provides a protective cavity for the brain.[1] The skull is composed of two parts: the cranium and the mandible. In humans, these two parts are the neurocranium and the viscerocranium or facial skeleton that includes the mandible as its largest bone. The skull forms the anterior most portion of the skeleton and is a product of cephalisation—housing the brain, and several sensory structures such as the eyes, ears, nose, and mouth.[2] In humans these sensory structures are part of the facial skeleton.  Functions of the skull include protection of the brain, fixing the distance between the eyes to allow stereoscopic vision, and fixing the position of the ears to enable sound localisation of the direction and distance of sounds. In some animals such as horned ungulates, the skull also has a defensive function by providing the mount (on the frontal bone) for the horns.  The Engli

In [0]:
import contractions
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords

clean_wiki_text = list(map(contractions.fix, wiki_text))
clean_wiki_text = [simple_preprocess(sentence, deacc = True) for sentence in clean_wiki_text]
'cute' in clean_wiki_text[5]

True

Notice the word "cute" is not in the model vocabulary.

In [0]:
model['cute']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


KeyError: ignored

# Re-training Model
- Make sure to use `update = True` parameter when building the vocab
- The training process use the same parameters, except for the `sentences` and you may use different `epochs`

In [0]:
model.build_vocab(clean_wiki_text, update = True)

2020-04-23 14:11:00,298 : INFO : collecting all words and their counts
2020-04-23 14:11:00,299 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-04-23 14:11:00,317 : INFO : collected 7254 word types from a corpus of 48577 raw words and 12 sentences
2020-04-23 14:11:00,318 : INFO : Updating model with new vocabulary
2020-04-23 14:11:00,327 : INFO : New added 714 unique words (8% of original 7968) and increased the count of 714 pre-existing words (8% of original 7968)
2020-04-23 14:11:00,333 : INFO : deleting the raw counts dictionary of 7254 items
2020-04-23 14:11:00,334 : INFO : sample=0.001 downsamples 112 most-common words
2020-04-23 14:11:00,335 : INFO : downsampling leaves estimated 43768 word corpus (127.3% of prior 34394)
2020-04-23 14:11:00,382 : INFO : estimated required memory for 1428 words and 300 dimensions: 4141200 bytes
2020-04-23 14:11:00,383 : INFO : updating layer weights


In [0]:
new_vocab = list(model.wv.vocab)
print(model) # vocab: 23190 -> 23207

Word2Vec(vocab=23207, size=300, alpha=0.005)


Our vocab gain new 17 words. Here is the list:

In [0]:
set(new_vocab) - set(old_vocab)

{'brow',
 'cocps',
 'cute',
 'cuteness',
 'enovid',
 'lolita',
 'lolitas',
 'mdna',
 'mestranol',
 'norethisterone',
 'noretynodrel',
 'pincus',
 'progestins',
 'searle',
 'taimyr',
 'ucl',
 'ybp'}

In [0]:
new_sentence = clean_wiki_text
callback = Callback()

model.train(new_sentence,
            total_examples = model.corpus_count,
            epochs = 5,
            compute_loss = True,
            callbacks = [callback]) 

2020-04-23 14:01:05,849 : WARNING : Effective 'alpha' higher than previous training cycles
2020-04-23 14:01:05,851 : INFO : training model with 3 workers on 23207 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=20 window=5
2020-04-23 14:01:06,765 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-04-23 14:01:06,810 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-04-23 14:01:06,973 : INFO : EPOCH 1 - PROGRESS: at 100.00% examples, 30407 words/s, in_qsize 0, out_qsize 1
2020-04-23 14:01:06,974 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-04-23 14:01:06,976 : INFO : EPOCH - 1 : training on 48577 raw words (33750 effective words) took 1.1s, 30334 effective words/s


Loss after epoch 1: 292107.4375


2020-04-23 14:01:07,797 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-04-23 14:01:07,937 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-04-23 14:01:08,120 : INFO : EPOCH 2 - PROGRESS: at 100.00% examples, 29762 words/s, in_qsize 0, out_qsize 1
2020-04-23 14:01:08,121 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-04-23 14:01:08,123 : INFO : EPOCH - 2 : training on 48577 raw words (33852 effective words) took 1.1s, 29676 effective words/s


Loss after epoch 2: 276003.5


2020-04-23 14:01:08,839 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-04-23 14:01:09,061 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-04-23 14:01:09,261 : INFO : EPOCH 3 - PROGRESS: at 100.00% examples, 29859 words/s, in_qsize 0, out_qsize 1
2020-04-23 14:01:09,262 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-04-23 14:01:09,262 : INFO : EPOCH - 3 : training on 48577 raw words (33771 effective words) took 1.1s, 29813 effective words/s
2020-04-23 14:01:09,263 : INFO : training on a 145731 raw words (101373 effective words) took 3.4s, 29722 effective words/s
2020-04-23 14:01:09,264 : WARNING : under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay


Loss after epoch 3: 266961.6875


(101373, 145731)

Check if the word "cute" already exist in our vocabulary.

In [0]:
model.wv["cute"]

array([ 7.4883504e-04, -2.6113447e-04,  2.3760644e-04,  9.3860098e-04,
       -1.4237677e-04,  1.2774217e-03, -3.3907688e-04,  4.6382050e-04,
        6.8105041e-04,  5.3802185e-04, -1.6347267e-03,  3.8733837e-04,
       -7.1973127e-04,  1.1154407e-03,  1.5528917e-03, -1.3417512e-03,
        9.2187803e-04,  5.1689404e-04, -8.2061382e-04, -5.0489570e-04,
       -9.9954013e-05,  6.3780806e-04, -2.4767834e-04,  1.0134879e-03,
        1.5266651e-03,  5.8589334e-04,  1.0786245e-03,  5.5645261e-04,
        1.5204248e-03,  9.8791206e-04, -6.3747400e-04, -9.5522183e-04,
        1.2925118e-03, -1.5111063e-03, -1.3946460e-03, -1.0777240e-03,
        3.8983164e-04,  1.0519883e-03, -1.2968043e-03,  8.2754489e-04,
        1.6474381e-05, -6.5679575e-04, -1.5853419e-03, -1.1392898e-03,
       -1.3087797e-03,  3.2610740e-04,  4.7327609e-05,  8.8706170e-04,
       -2.8948160e-04,  8.5462432e-04, -1.0303415e-03, -5.9961405e-04,
       -1.6496980e-03,  1.2708758e-03,  4.7012491e-04, -2.5329166e-04,
      